<a href="https://colab.research.google.com/github/appyjais/Text-Classification-and-summary/blob/main/NGram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
from nltk import word_tokenize
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk import ngrams
stop = stopwords.words('english')
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = '/content/drive/MyDrive/Assignment/securly/dataset/dataset/train.csv/train.csv'
df = pd.read_csv(path)

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
tags = ['toxic', 'severe_toxic','obscene', 'threat', 'insult', 'identity_hate']

In [5]:
def preprocess(df):
    df['preprocess'] = df.apply(lambda row: row['comment_text'].replace("\n"," "), axis=1) #removes new line character
    df['preprocess'] = df.apply(lambda row: re.sub('[^A-Za-z\' ]+', '',row['preprocess']).lower(), axis=1)
    df['preprocess'] = df['preprocess'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df['preprocess'] = df.apply(lambda row: re.sub('  +', ' ',row['preprocess']).strip(), axis=1)
    df["preprocess1"] = df.apply(lambda x: x["comment_text"] if len(x["preprocess"])==0 else x['preprocess'], axis=1)


In [6]:
preprocess(df)

In [7]:
df['ngram1'] = df.apply(lambda x: list(ngrams(x['preprocess1'].split(),1)), axis=1)
df['ngram2'] = df.apply(lambda x: list(ngrams(x['preprocess1'].split(),2)), axis=1)
df['ngram3'] = df.apply(lambda x: list(ngrams(x['preprocess1'].split(),3)), axis=1)
#df['ngram4'] = df.apply(lambda x: list(ngrams(x['preprocess1'].split(),4)), axis=1)
#df['ngram5'] = df.apply(lambda x: list(ngrams(x['preprocess1'].split(),5)), axis=1)
#df['ngram'] = df.apply(lambda x: x['ngram1']+x['ngram2']+x['ngram3']+x['ngram4']+x['ngram5'], axis=1)
df['ngram'] = df.apply(lambda x: x['ngram1']+x['ngram2']+x['ngram3'], axis=1)
l = ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'ngram']
df = df[l].explode('ngram')
df['ngram'] = df.apply(lambda x: ' '.join(t for t in x['ngram']), axis=1 )
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: generator 'ngrams' raised StopIteration
  This is separate from the ipykernel package so we can avoid doing imports until


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,ngram
0,0000997932d777bf,0,0,0,0,0,0,explanation
0,0000997932d777bf,0,0,0,0,0,0,edits
0,0000997932d777bf,0,0,0,0,0,0,made
0,0000997932d777bf,0,0,0,0,0,0,username
0,0000997932d777bf,0,0,0,0,0,0,hardcore


In [60]:
df[df['id']=='0007e25b2121310b']

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,ngram
16,0007e25b2121310b,1,0,0,0,0,0,bye
16,0007e25b2121310b,1,0,0,0,0,0,look
16,0007e25b2121310b,1,0,0,0,0,0,come
16,0007e25b2121310b,1,0,0,0,0,0,think
16,0007e25b2121310b,1,0,0,0,0,0,comming
16,0007e25b2121310b,1,0,0,0,0,0,back
16,0007e25b2121310b,1,0,0,0,0,0,tosser
16,0007e25b2121310b,1,0,0,0,0,0,bye look
16,0007e25b2121310b,1,0,0,0,0,0,look come
16,0007e25b2121310b,1,0,0,0,0,0,come think


In [38]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, Embedding, Input
from keras import metrics
import keras

In [32]:
traind=df[["id","ngram"]]
train_label=df[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]
tokenizer = Tokenizer(num_words = 20000)
tokenizer.fit_on_texts(traind.ngram)
train_final = tokenizer.texts_to_sequences(traind.ngram)
traind['ngram']=pad_sequences(train_final, maxlen=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
print(traind.shape,train_label.shape)

(15739832, 2) (15739832, 6)


In [34]:
max_features = 20000
embedding_dims = 32

In [35]:
model = Sequential()
model.add(Embedding(max_features, embedding_dims))
model.add(LSTM(units = embedding_dims, dropout = 0.2, recurrent_dropout = 0.2,return_sequences=True))
model.add(LSTM(units = embedding_dims, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(units = 6, activation = 'sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          640000    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 32)          8320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 656,838
Trainable params: 656,838
Non-trainable params: 0
_________________________________________________________________


In [40]:
from sklearn.model_selection import train_test_split
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [metrics.AUC(),metrics.Recall(), metrics.Accuracy(), metrics.Precision()])
x_train, x_val, y_train, y_val = train_test_split(traind, train_label, shuffle = True, random_state = 123)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
model.fit(x_train['ngram'], y_train, batch_size = 4096, epochs = 1, validation_data = (x_val['ngram'], y_val))

(11804874, 2) (11804874, 6) (3934958, 2) (3934958, 6)
2883/2883 [==============================] - 376s 128ms/step - loss: 0.1853 - auc_2: 0.6851 - recall_2: 0.0037 - accuracy: 0.0000e+00 - precision_2: 0.0398 - val_loss: 0.1247 - val_auc_2: 0.7354 - val_recall_2: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_precision_2: 0.0000e+00


In [41]:
model.save('Text_Classification_ngram')

INFO:tensorflow:Assets written to: Text_Classification_ngram/assets


In [42]:
model = keras.models.load_model('Text_Classification_ngram')

In [44]:
pred = model.predict(x_val['ngram'])

tags = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

In [45]:
pred.shape

(3934958, 6)

In [46]:
pred

array([[0.07959169, 0.01409069, 0.04272494, 0.00256628, 0.03771845,
        0.00832018],
       [0.07959169, 0.01409069, 0.04272494, 0.00256628, 0.03771845,
        0.00832018],
       [0.07959169, 0.01409069, 0.04272494, 0.00256628, 0.03771845,
        0.00832018],
       ...,
       [0.07959169, 0.01409069, 0.04272494, 0.00256628, 0.03771845,
        0.00832018],
       [0.07959169, 0.01409069, 0.04272494, 0.00256634, 0.03771845,
        0.00832018],
       [0.07959169, 0.01409069, 0.04272494, 0.00256634, 0.03771845,
        0.00832018]], dtype=float32)

In [55]:
pred_df = pd.DataFrame(pred, columns = tags)

In [53]:
from sklearn.metrics import f1_score

N = 50
threshold = [n / N for n in range(1,N)]
threshold_max = dict(zip(tags,[0]*6))
threshold_argmax = dict(zip(tags,[0]*6))
for i in tags:
  for t in threshold:
    pred_df['tmp'] = (pred_df[i]>= t).astype(int)
    score = f1_score(y_val[i], pred_df['tmp'])
    if score==0:
      break
    if(score > threshold_max[i]):
      threshold_max[i] = score
      threshold_argmax[i] = t

    print("%s : f1 score for threshold %0.2f :   %0.3f" %(i, t, score ))
pred_df.drop(columns  = 'tmp', inplace=True)
print(threshold_max)
print(threshold_argmax)

toxic : f1 score for threshold 0.02 :   0.143
toxic : f1 score for threshold 0.04 :   0.143
toxic : f1 score for threshold 0.06 :   0.143
toxic : f1 score for threshold 0.08 :   0.143
toxic : f1 score for threshold 0.10 :   0.143
toxic : f1 score for threshold 0.12 :   0.143
toxic : f1 score for threshold 0.14 :   0.143
toxic : f1 score for threshold 0.16 :   0.143
toxic : f1 score for threshold 0.18 :   0.143
toxic : f1 score for threshold 0.20 :   0.143
toxic : f1 score for threshold 0.22 :   0.143
toxic : f1 score for threshold 0.24 :   0.143
toxic : f1 score for threshold 0.26 :   0.143
toxic : f1 score for threshold 0.28 :   0.143
toxic : f1 score for threshold 0.30 :   0.143
toxic : f1 score for threshold 0.32 :   0.143
toxic : f1 score for threshold 0.34 :   0.143
toxic : f1 score for threshold 0.36 :   0.143
toxic : f1 score for threshold 0.38 :   0.143
toxic : f1 score for threshold 0.40 :   0.143
toxic : f1 score for threshold 0.42 :   0.143
toxic : f1 score for threshold 0.4

In [52]:
for i in tags:
  pred_df[i] = (pred_df[i]>= threshold_argmax[i]).astype(int)

In [57]:
pred_df.tail()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
3934953,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934954,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934955,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934956,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934957,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832


In [58]:
pred_df.tail()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
3934953,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934954,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934955,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934956,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
3934957,0.079592,0.014091,0.042725,0.002566,0.037718,0.00832
